In [1]:
# ! pip install fastparquet

In [2]:
import pandas as pd
from tqdm import tqdm
import swifter
import numpy as np
from glob import glob
import json
import gc


In [3]:
train = pd.read_parquet("../data_phase1/train.parquet")
val = pd.read_parquet("../data_phase1/validation.parquet")


In [4]:
train[["context_type"]] = train[["context_type"]].fillna(value="NA")
val[["context_type"]] = val[["context_type"]].fillna(value="NA")


In [5]:
train.head(2)

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,context_type,context_value,product_price,week,week_day,is_click
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,designer_id,e5315dbea15a033bc6974a0bccf5fae4a017648bcd92ab...,0.000263,7,6,0
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,product_id,61205c20046f2688cb7ed03cad29d5a5dbdc360ff48290...,0.012966,6,5,0


In [6]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [7]:
train = reduce_mem_usage(train)
val = reduce_mem_usage(val)


Memory usage of dataframe is 428.22 MB
Memory usage after optimization is: 175.85 MB
Decreased by 58.9%
Memory usage of dataframe is 78.64 MB
Memory usage after optimization is: 47.95 MB
Decreased by 39.0%


In [8]:
gc.collect()

521

In [9]:
train.shape, val.shape

((3507990, 16), (687192, 15))

In [10]:
train.memory_usage().sum()/(1024*1024*1024), val.memory_usage().sum()/(1024*1024*1024)

(0.1717264335602522, 0.04682622849941254)

In [11]:
full_df = pd.concat([train, val], 0)


In [12]:
d = full_df.groupby(["user_id"]).agg({"session_id": "nunique"}).reset_index()
d[d["session_id"]==1].shape

(158194, 2)

In [13]:
single_session_users = set(d.user_id.unique().tolist())

In [14]:
all_users = full_df.user_id.unique().tolist()

In [15]:
dic = {"user_id": [], "single_session_user_flag": []}

for user in tqdm(all_users):
    dic["user_id"].append(user)
    if user in single_session_users:
        dic["single_session_user_flag"].append(1)
    else:
        dic["single_session_user_flag"].append(0)
        

100%|██████████| 219035/219035 [00:00<00:00, 2027470.01it/s]


In [16]:
single_session_user_df = pd.DataFrame(dic, columns=["user_id", "single_session_user_flag"])
single_session_user_df.head(3)

,user_id,single_session_user_flag
0,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,1
1,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,1
2,90ea15d8d96a9d3e7ab463d990e5f4565cac9477498d37...,1


In [17]:
single_session_user_df.user_id.nunique(), single_session_user_df.shape

(219035, (219035, 2))

In [18]:
full_df['days_elapsed'] = (full_df['week'].astype(int)-1)*7 + full_df['week_day'].astype(int)
full_df.head(2)

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,context_type,context_value,product_price,week,week_day,is_click,days_elapsed
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,designer_id,e5315dbea15a033bc6974a0bccf5fae4a017648bcd92ab...,0.000263,7,6,0.0,48
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,product_id,61205c20046f2688cb7ed03cad29d5a5dbdc360ff48290...,0.012962,6,5,0.0,40


In [19]:
first_user_action_days_elapsed = pd.DataFrame(full_df.sort_values(["user_id", "days_elapsed"]).groupby(["user_id"])["days_elapsed"].first()).reset_index()
first_user_action_days_elapsed = first_user_action_days_elapsed.rename(columns={"days_elapsed": "first_user_action_days_elapsed"})
first_user_action_days_elapsed.head()



,user_id,first_user_action_days_elapsed
0,00001039003fd7fbc902f30fb6d78eaa5176f7291cba9a...,9
1,000032873c3c3fbab4ebc172399290d56247cbc1ef692c...,28
2,0000511b406188f830c5fbf3b3be20b6883ec872e418a1...,31
3,00008e290e495a3980448ae65d3de9d8857ab945949cc0...,13
4,0000b8fe24d60b23e0c70707e21eba6e3d28b45a85060b...,17


In [20]:
first_user_action_days_elapsed.user_id.nunique(), len(first_user_action_days_elapsed)

(219035, 219035)

In [21]:
len(list(set(first_user_action_days_elapsed.user_id.tolist()).intersection(set(single_session_user_df.user_id.tolist()))))


219035

In [22]:
user_feature_df = first_user_action_days_elapsed.merge(single_session_user_df, on=["user_id"], how="inner")
user_feature_df.head(3)

,user_id,first_user_action_days_elapsed,single_session_user_flag
0,00001039003fd7fbc902f30fb6d78eaa5176f7291cba9a...,9,1
1,000032873c3c3fbab4ebc172399290d56247cbc1ef692c...,28,1
2,0000511b406188f830c5fbf3b3be20b6883ec872e418a1...,31,1


In [23]:
user_feature_df.shape

(219035, 3)

In [24]:
# user_feature_df.to_csv("../preprocessed_data/user_time_features.csv", index=False)

# Non-click

In [25]:
full_df = full_df[["query_id", "session_id", "user_id", "product_id", "user_tier", "product_price"]]

In [26]:
attribute_df = pd.read_parquet("../data_phase1/attributes.parquet")


In [27]:
attribute_df = attribute_df[["product_id", "start_online_date"]]

In [28]:
full_df = full_df.merge(attribute_df, on=["product_id"], how="left")


In [29]:
full_df.head(2)

,query_id,session_id,user_id,product_id,user_tier,product_price,start_online_date
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,0.000263,570.041667
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,0.012962,208.041667


In [33]:
user_tier_mean_price = full_df.groupby(["user_tier"]).agg({"product_price": "mean"}).reset_index()
user_tier_mean_price = user_tier_mean_price.rename(columns={"product_price": "user_tier_mean_product_price"})

user_tier_max_price = full_df.groupby(["user_tier"]).agg({"product_price": "max"}).reset_index()
user_tier_max_price = user_tier_max_price.rename(columns={"product_price": "user_tier_max_product_price"})
user_tier_max_price = user_tier_mean_price.merge(user_tier_max_price, on=["user_tier"], how="inner")

user_tier_min_price = full_df.groupby(["user_tier"]).agg({"product_price": "min"}).reset_index()
user_tier_min_price = user_tier_min_price.rename(columns={"product_price": "user_tier_min_product_price"})
user_tier_min_price = user_tier_max_price.merge(user_tier_min_price, on=["user_tier"], how="inner")

user_tier_min_price.head()

,user_tier,user_tier_mean_product_price,user_tier_max_product_price,user_tier_min_product_price
0,747e44b477c7693ebc24bee9538cad9155ba5132eeb81f...,0.001414,0.597656,0.000008
1,b366ccda4ef57a5cdc3f1c5afa6147dd30d0849bb88613...,0.001095,0.877930,0.000006
2,be303dabea69281312bc30730bcef52abb9d27331f2809...,0.001064,0.877930,0.000000
3,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,0.001194,0.847656,0.000007
4,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,0.001904,1.000000,0.000000


In [34]:
user_tier_mean_start_online_date = full_df.groupby(["user_tier"]).agg({"start_online_date": "mean"}).reset_index()
user_tier_mean_start_online_date = user_tier_mean_start_online_date.rename(columns={"start_online_date": "user_tier_mean_product_start_online_date"})

user_tier_max_start_online_date = full_df.groupby(["user_tier"]).agg({"start_online_date": "max"}).reset_index()
user_tier_max_start_online_date = user_tier_max_start_online_date.rename(columns={"start_online_date": "user_tier_max_product_start_online_date"})
user_tier_max_start_online_date = user_tier_mean_start_online_date.merge(user_tier_max_start_online_date, on=["user_tier"], how="inner")

user_tier_min_start_online_date = full_df.groupby(["user_tier"]).agg({"start_online_date": "min"}).reset_index()
user_tier_min_start_online_date = user_tier_min_start_online_date.rename(columns={"start_online_date": "user_tier_min_product_start_online_date"})
user_tier_min_start_online_date = user_tier_max_start_online_date.merge(user_tier_min_start_online_date, on=["user_tier"], how="inner")

user_tier_min_start_online_date.head()


,user_tier,user_tier_mean_product_start_online_date,user_tier_max_product_start_online_date,user_tier_min_product_start_online_date
0,747e44b477c7693ebc24bee9538cad9155ba5132eeb81f...,357.263766,3214.041667,0.0
1,b366ccda4ef57a5cdc3f1c5afa6147dd30d0849bb88613...,380.305476,3214.041667,0.0
2,be303dabea69281312bc30730bcef52abb9d27331f2809...,386.106489,3214.041667,0.0
3,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,362.882202,3214.041667,118.0
4,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,425.663298,3214.041667,0.0


# Click

In [37]:
click_data = train[train["is_click"]==1]
click_data = click_data[["query_id", "session_id", "user_id", "product_id", "user_tier", "product_price"]]
click_data = click_data.merge(attribute_df, on=["product_id"], how="left")
click_data.head(2)


,query_id,session_id,user_id,product_id,user_tier,product_price,start_online_date
0,207454e43755700a650c30fa0b655226e4a6d4c3a6fde2...,ec2877a9ea2383289d46b7ad8a5208a59c8e67d3783edb...,c2a9051f311a1f5a116a5b990c01421db70e17ce7b1fbd...,49463bcf36dd614f33ddd3acd6312e79a538301d5fbbbc...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,0.007858,272.041667
1,aa277f6a6bb9138964f3b75b32cc9bc8a7695a2350a4ba...,e2431d5d6e15857a6d901be25fcd8ac733b3424609f1ee...,8c895f8be59c089f91fea0fce424e66189b973246fcb4a...,feddd77c4d8453645c8bb9bdc45d8e6f1abfda041db06d...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,0.000201,163.041667


In [39]:
user_tier_mean_click_price = click_data.groupby(["user_tier"]).agg({"product_price": "mean"}).reset_index()
user_tier_mean_click_price = user_tier_mean_click_price.rename(columns={"product_price": "user_tier_mean_click_product_price"})

user_tier_max_click_price = click_data.groupby(["user_tier"]).agg({"product_price": "max"}).reset_index()
user_tier_max_click_price = user_tier_max_click_price.rename(columns={"product_price": "user_tier_max_click_product_price"})
user_tier_max_click_price = user_tier_mean_click_price.merge(user_tier_max_click_price, on=["user_tier"], how="inner")

user_tier_min_click_price = click_data.groupby(["user_tier"]).agg({"product_price": "min"}).reset_index()
user_tier_min_click_price = user_tier_min_click_price.rename(columns={"product_price": "user_tier_min_click_product_price"})
user_tier_min_click_price = user_tier_max_click_price.merge(user_tier_min_click_price, on=["user_tier"], how="inner")

user_tier_min_click_price.head()


,user_tier,user_tier_mean_click_product_price,user_tier_max_click_product_price,user_tier_min_click_product_price
0,747e44b477c7693ebc24bee9538cad9155ba5132eeb81f...,0.001503,0.597656,0.000016
1,b366ccda4ef57a5cdc3f1c5afa6147dd30d0849bb88613...,0.001082,0.877930,0.000008
2,be303dabea69281312bc30730bcef52abb9d27331f2809...,0.001056,0.867188,0.000008
3,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,0.001158,0.436035,0.000016
4,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,0.001987,0.934082,0.000003


In [42]:
user_tier_mean_click_start_online_date = click_data.groupby(["user_tier"]).agg({"start_online_date": "mean"}).reset_index()
user_tier_mean_click_start_online_date = user_tier_mean_click_start_online_date.rename(columns={"start_online_date": "user_tier_mean_click_product_start_online_date"})

user_tier_max_click_start_online_date = click_data.groupby(["user_tier"]).agg({"start_online_date": "max"}).reset_index()
user_tier_max_click_start_online_date = user_tier_max_click_start_online_date.rename(columns={"start_online_date": "user_tier_max_click_product_start_online_date"})
user_tier_max_click_start_online_date = user_tier_mean_click_start_online_date.merge(user_tier_max_click_start_online_date, on=["user_tier"], how="inner")

user_tier_min_click_start_online_date = click_data.groupby(["user_tier"]).agg({"start_online_date": "min"}).reset_index()
user_tier_min_click_start_online_date = user_tier_min_click_start_online_date.rename(columns={"start_online_date": "user_tier_min_click_product_start_online_date"})
user_tier_min_click_start_online_date = user_tier_max_click_start_online_date.merge(user_tier_min_click_start_online_date, on=["user_tier"], how="inner")

user_tier_min_click_start_online_date.head()


,user_tier,user_tier_mean_click_product_start_online_date,user_tier_max_click_product_start_online_date,user_tier_min_click_product_start_online_date
0,747e44b477c7693ebc24bee9538cad9155ba5132eeb81f...,355.391213,3214.041667,126.0
1,b366ccda4ef57a5cdc3f1c5afa6147dd30d0849bb88613...,378.023500,3214.041667,125.0
2,be303dabea69281312bc30730bcef52abb9d27331f2809...,385.404646,3214.041667,125.0
3,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,357.790215,3214.041667,125.0
4,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,426.511957,3214.041667,120.0


In [44]:
merged_user_tier_df = user_tier_min_price.merge(user_tier_min_start_online_date, on="user_tier", how="left")
merged_user_tier_df = merged_user_tier_df.merge(user_tier_min_click_price, on="user_tier", how="left")
merged_user_tier_df = merged_user_tier_df.merge(user_tier_min_click_start_online_date, on="user_tier", how="left")
merged_user_tier_df.head()


,user_tier,user_tier_mean_product_price,user_tier_max_product_price,user_tier_min_product_price,user_tier_mean_product_start_online_date,user_tier_max_product_start_online_date,user_tier_min_product_start_online_date,user_tier_mean_click_product_price,user_tier_max_click_product_price,user_tier_min_click_product_price,user_tier_mean_click_product_start_online_date,user_tier_max_click_product_start_online_date,user_tier_min_click_product_start_online_date
0,747e44b477c7693ebc24bee9538cad9155ba5132eeb81f...,0.001414,0.597656,0.000008,357.263766,3214.041667,0.0,0.001503,0.597656,0.000016,355.391213,3214.041667,126.0
1,b366ccda4ef57a5cdc3f1c5afa6147dd30d0849bb88613...,0.001095,0.877930,0.000006,380.305476,3214.041667,0.0,0.001082,0.877930,0.000008,378.023500,3214.041667,125.0
2,be303dabea69281312bc30730bcef52abb9d27331f2809...,0.001064,0.877930,0.000000,386.106489,3214.041667,0.0,0.001056,0.867188,0.000008,385.404646,3214.041667,125.0
3,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,0.001194,0.847656,0.000007,362.882202,3214.041667,118.0,0.001158,0.436035,0.000016,357.790215,3214.041667,125.0
4,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,0.001904,1.000000,0.000000,425.663298,3214.041667,0.0,0.001987,0.934082,0.000003,426.511957,3214.041667,120.0


In [45]:
# merged_user_tier_df.to_csv("../preprocessed_data/user_tier_features.csv", index=False)
